## Задача 1: кластеризация запросов и снижение размерности данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from collections import defaultdict
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.metrics import silhouette_score, silhouette_samples, rand_score, adjusted_rand_score, homogeneity_completeness_v_measure
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ru_stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Данные:** датасет amazon_queries.csv

**Основное задание:**



**1.   Выполните предобработку датасета. Подумайте, нужно ли делить данные на тренировочные и тестовые;**

In [ ]:
df = pd.read_csv('amazon_queries.csv')
df

,query
0,motorcycle camping tent
1,samson wireless microphone system
2,iphone xs max with ring holder
3,justice league blu ray
4,funny golf shirts for men
...,...
97325,bat skeleton
97326,jigsaw puzzles 500 pieces for adults famous pa...
97327,squidward nike shoes
97328,travis scott posters for room


In [ ]:
queries = list(set(df['query'].tolist()))

#queries

In [ ]:
vectors = TfidfVectorizer().fit_transform(queries)

# делить данные на тренировочные и тестовые не требуется, т.к. у нас нет независимой переменной

**2.   Выполните кластеризацию методом К средних**

In [ ]:
k_means_ = KMeans(n_clusters=10).fit(vectors)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**3.   Посмотрите на запросы каждого кластера**

In [ ]:
texts_ = defaultdict(list)

In [ ]:
for text, cluster in zip(queries, k_means_.labels_):
  texts_[cluster].append(text)

In [ ]:
texts_

In [ ]:
texts_.keys()


dict_keys([1, 0, 5, 3, 7, 2, 9, 8, 4, 6])

In [ ]:
# @title
texts_[6][:5]

['its colombai not columbia',
 'the journey of not knowing',
 '1 hour twist timer not photo electric electrical outlets',
 'do not roll down window stickers',
 'the not a cat cat']

**4.   Снизьте размерность данных при помощи любого известного вам метода (подумайте, какой метод здесь будет наиболее подходящим)**

In [ ]:
# наиболее подходящим будет сингулярное разложение

model = TruncatedSVD(n_components=100)

reduced_matrix = model.fit_transform(vectors)

In [ ]:
reduced_matrix.shape

(97330, 100)

**5.   Кластеризуйте данные сниженной размерности такой же моделью, как и в шаге 2**

In [ ]:
k_means_reduced = KMeans(n_clusters=10).fit(reduced_matrix)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**6.   При помощи метрик, не требующих наличия тестового множества, сравните качество кластеризации на исходных данных и данных сниженной размерности.**

In [ ]:
silhouette_ = silhouette_score(vectors, k_means_.labels_)

In [ ]:
silhouette_

0.007106701673117446

In [ ]:
silhouette_reduced = silhouette_score(vectors, k_means_reduced.labels_)

In [ ]:
silhouette_reduced

0.0075320878909118354

## Задача 2: тематическое моделирование методом LDA

**Данные:** АРХИВ лемматизированных русских текстов ruslit_lemmas.zip

**Основное задание:**

**1. Достаньте тексты и (на всякий случай) их названия из архива**

In [ ]:
from zipfile import ZipFile

path_to_archive = 'ruslit_lemmas.zip'

def texts_from_zip(path_to_archive):
    texts = []
    with ZipFile(path_to_archive) as zip_archive:
        for name in zip_archive.namelist():
            if '.txt' in name:
                text = zip_archive.read(name)
                text = text.decode('utf-8')
                texts.append(text)
    return texts

def get_filenames(path_to_archive):

    names = []

    with ZipFile(path_to_archive) as zip_archive:
        for name in zip_archive.namelist():
            if '.txt' in name:
                names.append(name.split('/')[1])

    return names

**2.   Векторизуйте тексты. Ограничьте количество используемых признаков до 6000 и используйте стоп-слова**

In [ ]:
names = get_filenames(path_to_archive)
texts = texts_from_zip(path_to_archive)

In [ ]:
tf_vectorizer = TfidfVectorizer(max_features = 6000,
                                stop_words = ru_stop_words)

In [ ]:
vectors = tf_vectorizer.fit_transform(texts)

**3.   Сделайте тематическое моделирование при помощи латентного размещения Дирихле. Число компонент выберите любое**

In [ ]:
lda = LatentDirichletAllocation(n_components=100, random_state=0)

In [ ]:
topic = lda.fit_transform(vectors)

**4.   Выведите топовые слова каждой компоненты.**

In [ ]:
feature_names = tf_vectorizer.get_feature_names_out()

top_features_per_topic = defaultdict(list)
n_top_words = 25

for topic_idx, topic in enumerate(lda.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    top_features_per_topic[topic_idx] = top_features

In [ ]:
for topic, features in top_features_per_topic.items():
    print(f"Topic {topic}: {features}")

Topic 0: ['ивановна', 'елена', 'нехлюдов', 'свобода', 'партия', 'неожиданно', 'рабство', 'матвеич', 'сиятельство', 'крокодил', 'одинаково', 'всеобщий', 'революция', 'обертываться', 'естественно', 'наставать', 'основывать', 'призрак', 'наедине', 'хохот', 'наклоняться', 'освобождаться', 'кофе', 'поведение', 'обходиться']
Topic 1: ['нина', 'библиотека', 'тогдашний', 'весть', 'добиваться', 'юность', 'германия', 'наносить', 'повторяться', 'всматриваться', 'элемент', 'пленный', 'шпага', 'восстанавливать', 'мрамор', 'приготовление', 'строй', 'огорчать', 'комендант', 'окликать', 'владелец', 'прокламация', 'истязание', 'шпион', 'вывозить']
Topic 2: ['германн', 'зеркало', 'весна', 'лизавета', 'графиня', 'ивановна', 'петя', 'седло', 'бурый', 'растопчин', 'румянец', 'чахоточный', 'гостья', 'дабы', 'алпатыч', 'смоленск', 'балашев', 'обоз', 'аптека', 'бородинский', 'дрянь', 'изображение', 'посматривать', 'буквально', 'ростов']
Topic 3: ['дергать', 'употребление', 'тянуть', 'рыбина', 'веселие', 'рват

## Задача 3: ансамблевое обучение

**Датасет:** данные о качестве воды BKB_WaterQualityData_2020084.csv.

**Основное задание:**


1.   **Выполните предобработку датасета.** Ваша зависимая переменная - Site_Id. Ваши независимые переменные: Salinity (ppt), Dissolved Oxygen (mg/L), pH (standard units), Secchi Depth (m), Water Depth (m), Water Temp (?C), AirTemp (C). Подумайте, надо ли что-то кодировать и/или разделять выборку на тренировочное и тестовое множество.

In [ ]:
df1 = pd.read_csv('BKB_WaterQualityData_2020084.csv')

df1

,Site_Id,Unit_Id,Read_Date,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp-Celsius,Air Temp (?F),Time (24:00),Field_Tech,DateVerified,WhoVerified,AirTemp (C),Year
0,Bay,NaN,1/3/1994,1.3,11.7,7.3,0.40,0.40,5.9,8.0,46.40,11:00,NaN,NaN,NaN,8.000000,1994
1,Bay,NaN,1/31/1994,1.5,12.0,7.4,0.20,0.35,3.0,2.6,36.68,11:30,NaN,NaN,NaN,2.600000,1994
2,Bay,NaN,2/7/1994,1.0,10.5,7.2,0.25,0.60,5.9,7.6,45.68,9:45,NaN,NaN,NaN,7.600000,1994
3,Bay,NaN,2/23/1994,1.0,10.1,7.4,0.35,0.50,10.0,2.7,36.86,NaN,NaN,NaN,NaN,2.700000,1994
4,Bay,NaN,2/28/1994,1.0,12.6,7.2,0.20,0.40,1.6,0.0,32.00,10:30,NaN,NaN,NaN,0.000000,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Bay,NaN,10/11/2018,1.9,5.0,7.0,4.00,1.20,25.0,NaN,78.00,09:30,Sue Poe,11/13/2019,Christine Folks,25.555556,2018
2367,Bay,NaN,10/24/2018,0.0,9.0,7.0,0.30,0.60,18.0,NaN,58.00,09:30,Sue Poe,11/13/2019,Christine Folks,14.444444,2018
2368,Bay,NaN,10/28/2018,0.9,2.9,7.0,0.40,0.90,13.0,NaN,49.00,09:20,Sue Poe,11/13/2019,Christine Folks,9.444444,2018
2369,Bay,NaN,11/7/2018,1.7,NaN,7.0,0.45,0.90,20.0,NaN,65.00,09:45,Sue Poe,11/13/2019,Christine Folks,18.333333,2018


In [ ]:
water_ = df1.drop(columns=['Unit_Id', 'Read_Date', 'Time (24:00)', 'Air Temp (?F)', 'Air Temp-Celsius', 'Year', 'Field_Tech', 'DateVerified', 'WhoVerified'])

In [ ]:
water_ = water_.dropna()

In [ ]:
water_.head()

,Site_Id,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),AirTemp (C)
0,Bay,1.3,11.7,7.3,0.40,0.40,5.9,8.0
1,Bay,1.5,12.0,7.4,0.20,0.35,3.0,2.6
2,Bay,1.0,10.5,7.2,0.25,0.60,5.9,7.6
3,Bay,1.0,10.1,7.4,0.35,0.50,10.0,2.7
4,Bay,1.0,12.6,7.2,0.20,0.40,1.6,0.0


In [ ]:
X_water = water_[[i for i in water_.columns if i != 'Site_Id']]
y_water = water_['Site_Id'].tolist()

In [ ]:
X_water_train, X_water_test, y_water_train, y_water_test = train_test_split(X_water, y_water, test_size=0.2, stratify=y_water)

In [ ]:
le = LabelEncoder()
le.fit(y_water_train)

LabelEncoder()

In [ ]:
y_water_train_enc = le.transform(y_water_train)
y_water_test_enc = le.transform(y_water_test)

**2.   Используйте модели ансамблевого обучения (как минимум, рандомный лес и градиентный бустинг), чтобы по данным о качестве воды предсказать место, откуда был взят образец. Оцените качество предсказаний.**

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_water_train, y_water_train_enc)

RandomForestClassifier()

In [ ]:
water_pred = clf.predict(X_water_test)
print(classification_report(y_water_test_enc, water_pred, target_names=le.classes_))

              precision    recall  f1-score   support

           A       0.37      0.22      0.27        32
           B       0.53      0.56      0.54        34
         Bay       0.79      0.89      0.83       136
           C       0.69      0.43      0.53        21
           D       0.81      0.83      0.82        41

    accuracy                           0.72       264
   macro avg       0.64      0.59      0.60       264
weighted avg       0.70      0.72      0.70       264



In [ ]:
gr = GradientBoostingClassifier(random_state=0)
gr.fit(X_water_train, y_water_train_enc)

GradientBoostingClassifier(random_state=0)

In [ ]:
gr_pred = gr.predict(X_water_test)
print(classification_report(y_water_test_enc, gr_pred, target_names=le.classes_))

              precision    recall  f1-score   support

           A       0.57      0.38      0.45        32
           B       0.61      0.50      0.55        34
         Bay       0.84      0.90      0.87       136
           C       0.43      0.57      0.49        21
           D       0.83      0.85      0.84        41

    accuracy                           0.75       264
   macro avg       0.66      0.64      0.64       264
weighted avg       0.74      0.75      0.74       264

